In [1]:
import numpy as np 
import matplotlib.pyplot as plt

%matplotlib inline

import shapefile
import pyproj


#! /usr/bin/env python3

# data libraries
import sys
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from netCDF4 import Dataset,num2date  # http://code.google.com/p/netcdf4-python/
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from matplotlib.path import Path

# plotting libraries
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.style as style
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point 
import cartopy.mpl.ticker as cticker
import glob
import pickle

In [2]:
def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

In [3]:
def cropping_function(path,path1):
# Lists of data pulled from paths to create new csv of cropped data
    fpath = list() # The path to location in my raid directory
    model_names = list() # The model name pulled from the file name of the data 
    modeltemps = list() # The temperature of the shelf for each year from model
    latitude = list()
    longitude = list()
    year = list()
    month = list()
    gridsize = list()
    temp_crop = list()
    newlat = list()
    newlon = list()
    spatial_avg = list()
    
    shelf = gpd.read_file(f'{path1}') 
    shelf.crs = "EPSG:3031"
    shelf = shelf.to_crs(epsg=4326)

    shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')
    
    shelf_lon = np.array(shelf_lon)
    shelf_lon = shelf_lon - 360*((shelf_lon)-np.abs(shelf_lon))/(2*np.abs(shelf_lon))
    
    shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')
    shelf_list = np.vstack([shelf_lat,shelf_lon]).T
    shelf_path = Path(shelf_list)
    
    #print(shelf_path)
    
    for file in glob.iglob(f'{path}/*.nc'): #iterating through files in path specified
        
        #Reading in Datafiles
        fpath.append(file)
        fname = file
        datain = Dataset(fname, 'r')
        
        ##Splitting The Path to label the models
        head_tail = os.path.split(file) #makes the path a variable
        mystring = str(head_tail[1]) #turns the variable into a string
        x = mystring.split(".", 5)#parsing the model name by periods
        model_names.append(x[1]) # the second section of string is the model name
        print(model_names)
        #Creating Variables
        var = datain.variables['tas'] #2meter surface temperture
        var = var[:,:,:] - 273.15 #converting the temperature from Kelvin to Celsius
        modeltemps.append(var)
        grid = var[1].size
        gridsize.append(grid)
        lat = datain.variables['latitude'][:] #latitude
        latitude.append(lat)
        lon = datain.variables['longitude'][:] #longitude
        longitude.append(lon)
        timevar = datain.variables['time'] #in days since 1850
        dtime = num2date(timevar[:],timevar.units) # The next two lines make the time variable readable
        ym = np.zeros((len(dtime),2), int)
        
        shelf_lon_model, shelf_lat_model = np.meshgrid(lon,lat)
        inds = shelf_path.contains_points(np.vstack([shelf_lat_model.ravel(), shelf_lon_model.ravel()]).T)
        
        i = 0
        
        newvar = list()
        var_spatial_avg = list()
        for i in range(len(var)):
            vartemp = var[i].reshape((1,len(inds)))
            newvar.append(vartemp[0][inds])
            var_avg = np.mean(vartemp[0][inds])
            var_spatial_avg.append(var_avg)
        
        temp_crop.append(newvar)
        spatial_avg.append(var_spatial_avg)
        
            
        lat_temp = shelf_lat_model.ravel()[inds]
        lon_temp = shelf_lon_model.ravel()[inds]
        newlat.append(lat_temp)
        newlon.append(lon_temp)
        

        ##Time Component
        for i in range(len(dtime)): #Creating an aray of years from the time variable
            ym[i,0] = dtime[i].year
            ym[i,1] = dtime[i].month
       
        year.append(ym[:,0])
        month.append(ym[:,1])
    
        df = pd.DataFrame(list(zip(fpath,model_names,gridsize,modeltemps,year,month,latitude,longitude,temp_crop,newlat,newlon,spatial_avg)),
                      columns = ['file path', 'Model','gridsize','temp','year','month','original latitude','original longitude', 'cropped temp','cropped lat',
                                 'cropped lon','temp spatial average'])
    
    return df

In [4]:
fpath = list() # The path to location in my raid directory
model_names = list() # The model name pulled from the file name of the data 
modeltemps = list() # The temperature of the shelf for each year from model
latitude = list()
longitude = list()
year = list()
month = list()
gridsize = list()
temp_crop = list()
newlat = list()
newlon = list()
spatial_avg = list()

path = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/yearly'
path1 = '/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Abbot.shp'

shelf = gpd.read_file(f'{path1}') 
shelf.crs = "EPSG:3031"
shelf = shelf.to_crs(epsg=4326)

shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')

shelf_lon = np.array(shelf_lon)
#shelf_lon = shelf_lon - 360*((shelf_lon)-np.abs(shelf_lon))/(2*np.abs(shelf_lon))

shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')
shelf_list = np.vstack([shelf_lat,shelf_lon]).T
shelf_path = Path(shelf_list)

for file in glob.iglob(f'{path}/*.nc'): #iterating through files in path specified

    #Reading in Datafiles
    fpath.append(file)
    fname = file
    datain = Dataset(fname, 'r')

    ##Splitting The Path to label the models
    head_tail = os.path.split(file) #makes the path a variable
    mystring = str(head_tail[1]) #turns the variable into a string
    x = mystring.split(".", 5)#parsing the model name by periods
    model_names.append(x[1]) # the second section of string is the model name
    print(model_names)
    #Creating Variables
    var = datain.variables['tas'] #2meter surface temperture
    var = var[:,:,:] - 273.15 #converting the temperature from Kelvin to Celsius
    modeltemps.append(var)
    grid = var[1].size
    gridsize.append(grid)
    lat = datain.variables['latitude'][:] #latitude
    latitude.append(lat)
    lon = datain.variables['longitude'][:] #longitude
    longitude.append(lon)
    timevar = datain.variables['time'] #in days since 1850
    dtime = num2date(timevar[:],timevar.units) # The next two lines make the time variable readable
    ym = np.zeros((len(dtime),2), int)

    shelf_lon_model, shelf_lat_model = np.meshgrid(lon,lat)
    inds = shelf_path.contains_points(np.vstack([shelf_lat_model.ravel(), shelf_lon_model.ravel()]).T)
    print(inds)

    i = 0

    newvar = list()
    var_spatial_avg = list()
    for i in range(len(var)):
        vartemp = var[i].reshape((1,len(inds)))
        newvar.append(vartemp[0][inds])
        #print(vartemp[0])
        var_avg = np.mean(vartemp[0][inds])
        var_spatial_avg.append(var_avg)

    temp_crop.append(newvar)
    print(var_spatial_avg)
    spatial_avg.append(var_spatial_avg)


    lat_temp = shelf_lat_model.ravel()[inds]
    lon_temp = shelf_lon_model.ravel()[inds]
    newlat.append(lat_temp)
    newlon.append(lon_temp)


    ##Time Component
    for i in range(len(dtime)): #Creating an aray of years from the time variable
        ym[i,0] = dtime[i].year
        ym[i,1] = dtime[i].month

    year.append(ym[:,0])
    month.append(ym[:,1])


df = pd.DataFrame(list(zip(fpath,model_names,gridsize,modeltemps,year,month,latitude,longitude,temp_crop,newlat,newlon,spatial_avg)),
                  columns = ['file path', 'Model','gridsize','temp','year','month','original latitude','original longitude', 'cropped temp','cropped lat',
                             'cropped lon','temp spatial average'])


['NorESM2-MM']
[ True  True False  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False

['NorESM2-MM', 'E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-1-1-ECA', 'E3SM-Project']
[ True  True False  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  

[ True  True False  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False  T

['NorESM2-MM', 'E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-1-1-ECA', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'MPI-ESM1-2-LR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR']
[ True  True False  True  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False  True  True
  True  True  True  T

[-12.608053, -13.296559, -14.519859, -12.119822, -13.630043, -15.554005, -14.366563, -14.425839, -14.166622, -14.113133, -15.311822, -13.371505, -14.851402, -14.442335, -13.489172, -13.6163, -13.595838, -14.505538, -15.842832, -12.640033, -12.238865, -12.253756, -13.283498, -11.367437, -12.368575, -11.628858, -12.19515, -15.258495, -14.96372, -11.830356, -14.034292, -14.315923, -13.801767, -13.67884, -15.26997, -14.637453, -14.271901, -12.931575, -12.878362, -14.617331, -16.323723, -15.006268, -12.925421, -14.500367, -14.143953, -15.428475, -14.678168, -14.317858, -13.70772, -14.463939, -15.0781555, -15.877996, -13.839141, -12.311852, -12.92074, -13.924592, -14.972743, -12.868282, -14.465993, -13.634161, -15.20747, -14.493745, -14.707044, -14.782626, -12.82744, -13.834402, -12.518922, -15.417979, -13.652027, -15.517163, -16.812092, -13.9671135, -15.4969845, -14.197326, -15.797439, -14.468709, -15.400019, -12.502372, -13.111189, -13.422534, -15.56182, -14.902786, -14.620389, -16.716326,

[-12.730701, -11.318395, -9.671221, -10.102617, -11.064468, -11.154821, -13.101688, -12.278828, -10.303095, -9.008356, -13.181004, -9.935375, -11.54513, -12.637704, -11.389424, -11.628646, -10.358183, -11.660336, -10.65008, -12.530068, -10.358827, -14.711413, -11.403293, -9.592561, -10.927021, -11.772814, -10.375358, -10.259934, -11.570121, -6.547333, -7.6330037, -10.495358, -9.93152, -15.2200165, -13.050211, -10.378147, -10.701138, -10.363271, -11.584727, -10.739441, -13.583842, -10.27331, -9.218619, -10.397119, -10.788873, -8.227869, -12.412924, -9.335246, -11.036137, -11.100354, -10.629761, -10.111877, -10.272668, -12.037801, -11.952417, -8.06468, -8.704885, -9.890103, -10.345984, -10.759672, -10.188554, -12.725525, -11.846031, -12.919378, -8.496668, -12.474337, -9.7963295, -11.853863, -10.961238, -10.948944, -9.106393, -9.065352, -7.4256525, -9.54559, -7.11649, -10.450874, -13.03867, -9.534095, -10.6080265, -10.953218, -10.555264, -11.258752, -12.939287, -9.142824, -9.484239, -10.8

[-6.8488564, -6.7524414, -6.1604514, -4.8534093, -7.392829, -7.3198113, -7.2833962, -5.803817, -7.514778, -8.178178, -7.419936, -5.9929767, -6.42545, -7.2367535, -4.813107, -5.315495, -5.831741, -6.649234, -7.377247, -6.779205, -6.673016, -7.1931705, -7.08419, -6.4181437, -7.4668183, -6.3504477, -6.9197655, -7.205463, -6.816757, -6.214968, -5.6255717, -6.7300234, -6.542876, -6.592296, -6.666888, -6.666516, -8.225238, -8.344938, -8.708541, -6.387927, -7.5967855, -7.4385524, -6.1762266, -6.801022, -8.190618, -5.843281, -6.7988143, -4.8044987, -6.623098, -6.3671823, -7.062786, -6.8659444, -7.5838494, -7.319172, -6.7214475, -5.418626, -6.4672785, -5.47506, -6.799811, -6.7501554, -6.8479714, -6.1106377, -6.569217, -6.747619, -8.898699, -7.815322, -6.775115, -6.6396484, -6.386856, -6.1076694, -6.315002, -5.3592854, -6.050055, -8.296665, -7.017242, -6.212498, -4.5528383, -6.7553496, -6.011245, -6.271907, -5.637344, -6.066013, -6.8125386, -6.676399, -8.129164, -7.3394957, -5.3732567, -5.762134

In [5]:
df

,file path,Model,gridsize,temp,year,month,original latitude,original longitude,cropped temp,cropped lat,cropped lon,temp spatial average
0,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,NorESM2-MM,285,"[[[-14.892242, -15.170868, -15.449493, -15.728...","[1849, 1850, 1851, 1852, 1853, 1854, 1855, 185...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-14.892242, -15.170868, -15.728119, -15.9261...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-18.356825, -16.395996, -14.848855, -13.86101..."
1,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,E3SM-1-1,285,"[[[-10.630127, -10.647766, -10.665436, -10.683...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-10.630127, -10.647766, -10.683075, -10.6730...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-13.611139, -13.315776, -12.215018, -14.56452..."
2,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,CAS-ESM2-0,285,"[[[-8.438019, -8.466522, -8.494965, -8.51413, ...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-8.438019, -8.466522, -8.51413, -8.527679, -...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-10.808054, -9.867529, -11.553769, -11.896316..."
3,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,KIOST-ESM,285,"[[[-4.6848145, -4.710602, -4.736389, -4.762176...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-4.6848145, -4.710602, -4.7621765, -4.787964...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-6.949247, -9.759517, -9.094064, -7.5870013, ..."
4,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,E3SM-1-1-ECA,285,"[[[-8.815155, -8.832397, -8.84967, -8.866913, ...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-8.815155, -8.832397, -8.866913, -8.855804, ...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-11.961982, -12.309593, -10.445205, -14.34577..."
5,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,E3SM-Project,285,"[[[-6.888397, -6.901947, -6.915497, -6.9290466...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-6.888397, -6.901947, -6.9290466, -6.920044,...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-9.50957, -9.724408, -12.095586, -11.956807, ..."
6,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,CAMS-CSM1-0,285,"[[[-9.682709, -9.79599, -9.909271, -10.0225525...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-9.682709, -9.79599, -10.0225525, -10.225311...","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-12.930931, -13.104834, -11.236541, -11.21789..."
7,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,MPI-ESM1-2-HR,285,"[[[-8.361359, -8.6128845, -9.010712, -9.408569...","[1850, 1851, 1852, 1853, 1854, 1855, 1856, 185...","[7, 7, 7, 7, 7, 7,

In [6]:
for file in glob.iglob('/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/amery/seasonal/*.nc'): #iterating through files in path specified
    fname = file
    datain = Dataset(fname, 'r')

    ##Splitting The Path to label the models
    head_tail = os.path.split(file) #makes the path a variable
    mystring = str(head_tail[1]) #turns the variable into a string
    x = mystring.split(".", 5)#parsing the model name by periods
    model_names.append(x[1]) # the second section of string is the model name
    print(model_names)
    #Creating Variables
    

['NorESM2-MM', 'E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-1-1-ECA', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'MPI-ESM1-2-LR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM', 'FGOALS-g3', 'ACCESS-ESM1-5', 'NorESM2-LM', 'ACESS-CM2', 'GFDL-ESM4', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'NESM3', 'MRI-ESM2-0', 'CanESM5', 'MIROC6', 'CESM2-WACCM', 'NESM3']
['NorESM2-MM', 'E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-1-1-ECA', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'MPI-ESM1-2-LR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM', 'FGOALS-g3', 'ACCESS-ESM1-5', 'NorESM2-LM', 'ACESS-CM2', 'GFDL-ESM4', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'NESM3', 'MRI-ESM2-0', 'CanESM5', 'MIROC6', 'CESM2-WACCM', 'NESM3', 'E3SM-1-1']
['NorESM2-MM', 'E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-1-1-ECA', 'E3SM-Project', 'CAM

In [7]:
rootdir = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/amery'
shapefile = '/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Amery.shp'

test = cropping_function(rootdir,shapefile)

UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
df